In [1]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [2]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: http://localhost:8001 - Authenticated: Yes


In [3]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 8e7b022e
{
    "uid": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "e93cc55f-e3cc-4020-aa2b-0711757c476f",
    "path": "/mnt/cluster/projects/99_jupyter-project",
    "path_type": "name",
    "company": "1575b6cf-41d3-4996-8dcc-73045c1d9539"
}


In [4]:
task_name = "Jupyter Simple VWT"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: ccb39753
{
    "uid": "ccb39753-ce42-4268-bc86-9bc81830888e",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:15.418619",
    "config": {
        "task_type": "parent",
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-vwt_1d9ee4a3",
    "parent": null,
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Jupyter VWT",
    "dependent_on": null,
    "priority": 100
}


In [5]:
stl_path = os.path.join(data_path, "mesh_vwt", "cfdGeom.stl")
file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/cfdGeom.stl/",
                            files=open(stl_path, 'r').read(), extra_headers={"content-type": "application/octet-stream"})
print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl


In [6]:
write_mesh_path = os.path.join(data_path, "mesh_vwt", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "mesh_vwt", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: 7a1d2473
{
    "uid": "7a1d2473-3830-49bc-ace7-529ece610413",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:15.719965",
    "config": {
        "cmd": "cfd.io.tasks.write_mesh",
        "case_dir": "foam",
        "base_mesh": {
            "type": "simpleBox",
            "cell_size": 2,
            "parameters": {
                "z0": "True",
                "square": "True"
            },
            "bounding_box": {
                "max": [
                    35,
                    35,
                    24
                ],
                "min": [
                    -35,
                    -35,
                    0
                ]
            }
        },
        "task_type": "magpy",
        "snappyhex_mesh": "dict data too large for JSON."
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-vwt_1d9ee4a3",
    "parent": {
        "uid": "ccb39753-ce42-4268-bc86-9bc81830888e

In [7]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": cpus,
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ]},
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: d21ac1c6
{
    "uid": "d21ac1c6-f24e-4e21-bd60-28fb5c24e354",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:16.163545",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": [
            2,
            1,
            1
        ],
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ],
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-vwt_1d9ee4a3",
    "parent": {
        "uid": "ccb39753-ce42-4268-bc86-9bc81830888e",
        "name": "Jupyter VWT"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Mesh",
    "dependent_on": {
        "uid": "7a1d2473-3830-49bc-ace7-529ece610413",
        "name": "Actions"
    },
    "priority": 100
}


In [8]:
payload_vwt = {
    "name": "VWT",
    "config": {
        "task_type": "cfd",
        "cmd": "wind_tunnel",
        "commands": [
            0,
            90,
            180,
            270
        ],
        "cpus": cpus,
        "iterations": {
            "init": 100,
            "run": 50
        }
    },
    "parent": parent_id,
    "dependent_on": mesh_id
}

vwt_task = client.create(f"/api/project/{project_id}/task/", payload_vwt)
vwt_id = vwt_task.get("uid")

print(f"VWT TASK: {vwt_id[:8]}")
print(json.dumps(vwt_task, indent=4))

VWT TASK: 5350fcb2
{
    "uid": "5350fcb2-71e0-405c-9e93-4cfe511743be",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:16.421542",
    "config": {
        "task_type": "parent",
        "cmd": "wind_tunnel",
        "commands": [
            0,
            90,
            180,
            270
        ],
        "cpus": [
            2,
            1,
            1
        ],
        "iterations": {
            "init": 100,
            "run": 50
        },
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-vwt_1d9ee4a3",
    "parent": {
        "uid": "ccb39753-ce42-4268-bc86-9bc81830888e",
        "name": "Jupyter VWT"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "VWT",
    "dependent_on": null,
    "priority": 100
}


In [9]:
payload = {
    "name": "Probing",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [1, 1, 1],
        "sets": [
            {"name": "set1",
             "points": [
                 [0, 0, 1],
                 [0, 1, 1]
             ]}
        ],
        "fields": ["U", "p"]
    },
    "parent": parent_id,
    "dependent_on": vwt_id
}
probe_task = client.create(f"/api/project/{project_id}/task/", payload)
probe_id = probe_task.get("uid")

print(f"PROBE TASK: {probe_id[:8]}")
print(json.dumps(probe_task, indent=4))


PROBE TASK: 8f66437f
{
    "uid": "8f66437f-3378-4d37-8d7f-a647a81c0b74",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:17.265873",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [
            1,
            1,
            1
        ],
        "sets": [
            {
                "name": "set1",
                "points": [
                    [
                        0,
                        0,
                        1
                    ],
                    [
                        0,
                        1,
                        1
                    ]
                ]
            }
        ],
        "fields": [
            "U",
            "p"
        ]
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-pro

In [10]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")